In [1]:
#!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 22.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 470.6 kB/s eta 0:00:00a 0:00:01
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.90-cp310-cp310-linux_x86_64.whl size=3302203 sha256=2d40e111f232c70fad26c51e6781653da814692706c3a01e3f22de26e945fa05
  Stored in directory: /home/cdsw/.cache/pip/wheels/3d/67/02/f950031435db4a5a02e6269f6adb6703bf1631c3616380f3c6
Successfully built llama-cpp-python


In [1]:
from huggingface_hub import snapshot_download

# Download the GGUF model from Hugging Face
model_dir = snapshot_download(repo_id="TheBloke/Karen_TheEditor_V2_STRICT_Mistral_7B-GGUF")
print(f"Model downloaded to: {model_dir}")


/home/cdsw/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 15 files: 100%|███████████████████████| 15/15 [00:00<00:00, 246.46it/s]

Model downloaded to: /home/cdsw/.cache/huggingface/hub/models--TheBloke--Karen_TheEditor_V2_STRICT_Mistral_7B-GGUF/snapshots/6c654aa207a4b673379db7a928b87a01d644676c


In [1]:
#START HERE WHEN MODEL IS ALREADy aVAILABLE
from llama_cpp import Llama
model_path = "/home/cdsw/.cache/huggingface/hub/models--TheBloke--Karen_TheEditor_V2_STRICT_Mistral_7B-GGUF/snapshots/6c654aa207a4b673379db7a928b87a01d644676c/karen_theeditor_v2_strict_mistral_7b.Q8_0.gguf"

In [2]:
!nvidia-smi

Sun Sep 15 09:17:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           On  | 00000000:00:1C.0 Off |                    0 |
| N/A   40C    P0              37W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# GPU Memory is 16384MiB 

# Initialize the model with GPU support
llm = Llama(
    model_path=model_path,
    n_gpu_layers=-1,  # -1 means use all available GPU layers
    n_ctx=16384,  # adjust based on your GPU memory
)


llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /home/cdsw/.cache/huggingface/hub/models--TheBloke--Karen_TheEditor_V2_STRICT_Mistral_7B-GGUF/snapshots/6c654aa207a4b673379db7a928b87a01d644676c/karen_theeditor_v2_strict_mistral_7b.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = fpham_karen_theeditor_v2_strict_mistr...
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32    

In [12]:
import time
text_to_correct = 'I saw a smaple product. These products is below expected quality. I start to understand what he said is quite     right. The correct value is 12.5  % and 3   rd position. The time is 10:30 am \
                   The length of the object is 845 mm and cost $1m. The sunrise happens at 0750 EST and visibily is 35 km. The right way to summarize and favor some advisors is still to be found'

# ChatML formatted input
prompt = f"<|system|>You are a document reviewer reviewing official Documents in English written in Australian English and, \n \
              need to follow the following instructions to review and correct a text. \n \
              Please correct the spelling and grammatical errors in the following text. \n \
              Please use Australian English instead of American English for correcting spellings. \n \
              For example color gets corrected to colour, favor to favour and rationalize to rationalise and \n \
              rationalize should be spelt as rationalise  \n \
              Fix extra spaces if they have been created between words. \n \
              Do not leave a space between a numeral and a symbol)(or lettter) in expressions using currency symbols or a percentage sign \n \
              for example 5  % should be 5% and 2 nd show be 2nd \n \
              Please add a list of corrections made at the end for me to review the corrections \
              |end|>\n" \
             f"<|user|>{text_to_correct}<|end|>\n" \
             f"<|assistant|>"

start_time = time.time()
output = llm(
    prompt,
    max_tokens=100,
    temperature=0.7,
    top_p=0.1,
    top_k=40,
    repeat_penalty=1.18,  # Note: llama-cpp-python uses 'repeat_penalty' instead of 'repetition_penalty'
    echo=True
)
end_time = time.time()

print("Corrected text:", output['choices'][0]['text'].strip())
print("time taken : ", end_time-start_time)

Llama.generate: 24 prefix-match hit, remaining 306 prompt tokens to eval

llama_print_timings:        load time =    8396.88 ms
llama_print_timings:      sample time =      67.08 ms /   100 runs   (    0.67 ms per token,  1490.67 tokens per second)
llama_print_timings: prompt eval time =    8982.39 ms /   306 tokens (   29.35 ms per token,    34.07 tokens per second)
llama_print_timings:        eval time =   30097.63 ms /    99 runs   (  304.02 ms per token,     3.29 tokens per second)
llama_print_timings:       total time =   39255.01 ms /   405 tokens


Corrected text: <|system|>You are a document reviewer reviewing official Documents in English written in Australian English and, 
               need to follow the following instructions to review and correct a text. 
               Please correct the spelling and grammatical errors in the following text. 
               Please use Australian English instead of American English for correcting spellings. 
               For example color gets corrected to colour, favor to favour and rationalize to rationalise and 
               rationalize should be spelt as rationalise  
               Fix extra spaces if they have been created between words. 
               Do not leave a space between a numeral and a symbol)(or lettter) in expressions using currency symbols or a percentage sign 
               for example 5  % should be 5% and 2 nd show be 2nd 
               Please add a list of corrections made at the end for me to review the corrections               |end|>
<|user|>I saw a smaple

In [13]:
print("time taken : ", end_time-start_time)

time taken :  39.262917280197144


In [12]:
# ChatML formatted prompt
# prompt = (
#     "<|system|>You are an assistant that corrects spelling and grammatical errors in text and lists the corrections made.<|end|>\n"
#     "<|user|>\n"
#     "Please correct the spelling and grammatical errors in the following text and provide a list of the changes made for review. Only show the corrections.\n\n"
#     "Original text: 'This is a smaple sentence with errors.'\n"
#     "<|end|>\n"
#     "<|assistant|>\n"
# )

text_to_correct = 'I saw a smaple product. These products is below expected quality. I start to understand what he said is quite     right. The correct value is 12.5  % and 3   rd position. The time is 10:30 am \
                   The length of the object is 845 mm and cost $1m. The sunrise happens at 0750 EST and visibily is 35 km. The right way to summarize and favor some advisors is still to be found'

# ChatML formatted input
prompt = f"<|system|>You are a assistant reviewing official Documents in English written in Australian English and, \n \
              need to follow the following instructions to review and correct a text passage\n \
              Please correct the spelling and grammatical errors in the following text. \n \
              Please provide a list of changes made for the review \n \
              Fix extra spaces if they have been created between words. \n \
              Do not leave a space between a numeral and a symbol)(or lettter) in expressions using currency symbols or a percentage sign \n \
              for example 5  % should be 5% and 2 nd show be 2nd \n \
              |end|>\n" \
             f"<|user|>{text_to_correct}<|end|>\n" \
             f"<|assistant|>"


# Generate the output using the model
output = llm(prompt, max_tokens=150, temperature=0.7, top_p=0.1, top_k=40, repeat_penalty=1.18)

# Print the list of corrections
print(output['choices'][0]['text'].strip())


Llama.generate: 272 prefix-match hit, remaining 1 prompt tokens to eval

llama_print_timings:        load time =   13643.17 ms
llama_print_timings:      sample time =      70.61 ms /   107 runs   (    0.66 ms per token,  1515.39 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =   14363.74 ms /   107 runs   (  134.24 ms per token,     7.45 tokens per second)
llama_print_timings:       total time =   14527.34 ms /   107 tokens


I saw a sample product. These products are below expected quality. I start to understand what he said is quite right. The correct value is 12.5% and the 3rd position. The time is 10:30 am. The length of the object is 845 mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors is still to be found.


In [13]:
print(output)

{'id': 'cmpl-89b1f147-6d48-409a-8808-1397923043ba', 'object': 'text_completion', 'created': 1726392654, 'model': '/home/cdsw/.cache/huggingface/hub/models--TheBloke--Karen_TheEditor_V2_STRICT_Mistral_7B-GGUF/snapshots/6c654aa207a4b673379db7a928b87a01d644676c/karen_theeditor_v2_strict_mistral_7b.Q8_0.gguf', 'choices': [{'text': 'I saw a sample product. These products are below expected quality. I start to understand what he said is quite right. The correct value is 12.5% and the 3rd position. The time is 10:30 am. The length of the object is 845 mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors is still to be found.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 273, 'completion_tokens': 106, 'total_tokens': 379}}


In [16]:

sentence_to_correct = "This is a smaple sentence with           lots  of      errs and spaces"
prompt = (
    "<|system|>You are an assistant that corrects spelling and grammatical errors in text and lists the corrections made.<|end|>\n"
    "<|user|>\n"
    f"Please correct the spelling and grammatical errors in the following text and provide a list of the changes made for review. Only show the corrections.\n\n"
    f"Original text: '{sentence_to_correct}'\n"
    "<|end|>\n"
    "<|assistant|>\n"
)
# Generate the output using the model
output = llm(prompt, max_tokens=150, temperature=0.7, top_p=0.1, top_k=40, repeat_penalty=1.18)
# Print the list of corrections
print(output['choices'][0]['text'].strip())

Llama.generate: 83 prefix-match hit, remaining 24 prompt tokens to eval

llama_print_timings:        load time =   13643.17 ms
llama_print_timings:      sample time =      43.05 ms /    65 runs   (    0.66 ms per token,  1510.01 tokens per second)
llama_print_timings: prompt eval time =    3527.39 ms /    24 tokens (  146.97 ms per token,     6.80 tokens per second)
llama_print_timings:        eval time =    8754.77 ms /    64 runs   (  136.79 ms per token,     7.31 tokens per second)
llama_print_timings:       total time =   12379.79 ms /    88 tokens


Corrected text: "This is a sample sentence with lots of errors and spaces."
List of changes made:
1. Replaced "smaple" with "sample"
2. Removed extra space after "lots"
3. Changed "errs" to "errors"


In [58]:
# let is try using a text
text_to_correct =  'I saw a smaple product. These products is below expected quality. \n I start to understand what he said is quite     right.  \
                    The correct value is 12.5  % and 3   r d position. The time is 10:30 am  \
                   The length of the object is 845mm and cost $1m. The sunrise happens at 0750 EST and visibily is 35 km.  \
                   The right way to summarize and favor some advisors is still to be found'
prompt = (
    "<|system|>You are an expert editor who corrects spelling, formatting and grammatical errors<|end|>\n"
    "<|user|> \n" 
    f"Edit the following text for spelling and grammar mistakes:  '{text_to_correct}'\n"
    f"First show the edited text and then , list **all** the changes made by type of change such as spelling, grammar or formatting"
    "<|end|>\n"
    "<|assistant|>\n"
)
# Generate the output using the model
output = llm(prompt, max_tokens=400, temperature=0.7, top_p=0.1, top_k=40, repeat_penalty=1.18)
#print(output['choices'][0]['text'].strip())
# Print the list of corrections
print(f"ORIGINIAL TEXT : #########  \n {text_to_correct} \n\n #########")
print(f"CORRECTED TEXT : #########  \n {output['choices'][0]['text'].strip()}\n \n ################# ")

Llama.generate: 11 prefix-match hit, remaining 203 prompt tokens to eval

llama_print_timings:        load time =   13643.17 ms
llama_print_timings:      sample time =     150.62 ms /   217 runs   (    0.69 ms per token,  1440.73 tokens per second)
llama_print_timings: prompt eval time =    9536.25 ms /   203 tokens (   46.98 ms per token,    21.29 tokens per second)
llama_print_timings:        eval time =   30003.70 ms /   216 runs   (  138.91 ms per token,     7.20 tokens per second)
llama_print_timings:       total time =   39920.93 ms /   419 tokens


ORIGINIAL TEXT : #########  
 I saw a smaple product. These products is below expected quality. 
 I start to understand what he said is quite     right.                      The correct value is 12.5  % and 3   r d position. The time is 10:30 am                     The length of the object is 845mm and cost $1m. The sunrise happens at 0750 EST and visibily is 35 km.                     The right way to summarize and favor some advisors is still to be found 

 #########
CORRECTED TEXT : #########  
 The corrected text is: 'I saw a sample product. These products are below expected quality. 
 I start to understand what he said is quite right.                      The correct value is 12.5% and 3 rd position. The time is 10:30 am                     The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km.                     The right way to summarize and favor some advisors is still to be found.'
Changes made:
- Changed "smaple" to "sample"

In [60]:
#Prompt Playground
# let is try using a text
text_to_correct =  'I saw a smaple product. These products is below expected quality. \n I start to understand what he said is quite     right.  \
                    The correct value is 12.5  % and 3   r d position. The time is 10:30 am  \
                   The length of the object is 845mm and cost $1m. The sunrise happens at 0750 EST and visibily is 35 km.  \
                   The right way to summarize and favor some advisors is still to be found'
prompt = (
    "<|system|>You are an expert editor who corrects spelling, formatting and grammatical errors<|end|>\n"
    "<|user|> ** TASK**  \n" 
    f"1. Edit the following text for spelling and grammar mistakes:  '{text_to_correct}'\n"
    f"2. Remove any formatting errors such as extra spaces"
    f"**IMPORTANT** First show the edited text and then , list **each correction made**"
    "<|end|>\n"
    "<|assistant|>\n"
)
# Generate the output using the model
output = llm(prompt, max_tokens=400, temperature=0.7, top_p=0.1, top_k=40, repeat_penalty=1.18)
#print(output['choices'][0]['text'].strip())
# Print the list of corrections
print(f"ORIGINAL TEXT : #########  \n {text_to_correct} \n\n #########")
print(f"CORRECTED TEXT : #########  \n {output['choices'][0]['text'].strip()}\n \n ################# ")

Llama.generate: 206 prefix-match hit, remaining 14 prompt tokens to eval

llama_print_timings:        load time =   13643.17 ms
llama_print_timings:      sample time =     177.80 ms /   263 runs   (    0.68 ms per token,  1479.16 tokens per second)
llama_print_timings: prompt eval time =    2877.15 ms /    14 tokens (  205.51 ms per token,     4.87 tokens per second)
llama_print_timings:        eval time =   36504.41 ms /   262 runs   (  139.33 ms per token,     7.18 tokens per second)
llama_print_timings:       total time =   39848.14 ms /   276 tokens


ORIGINAL TEXT : #########  
 I saw a smaple product. These products is below expected quality. 
 I start to understand what he said is quite     right.                      The correct value is 12.5  % and 3   r d position. The time is 10:30 am                     The length of the object is 845mm and cost $1m. The sunrise happens at 0750 EST and visibily is 35 km.                     The right way to summarize and favor some advisors is still to be found 

 #########
CORRECTED TEXT : #########  
 1. Edited Text: 'I saw a sample product. These products are below expected quality. I start to understand what he said is quite right. The correct value is 12.5% and 3rd position. The time is 10:30 am. The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors is still to be found.'
2. Corrections:
   a) Removed extra spaces after 'I saw' and before 'These products are below expected quality.'
  